In [ ]:
import numpy as np
import pandas as pd
import nd2reader
import matplotlib.pyplot as plt
import holoviews as hv
import skimage.filters
import skimage.feature
import scipy.ndimage
import peakutils
from tqdm import tnrange, tqdm_notebook
import dask
import dask.array as da
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from cytoolz import partial, compose
from itertools import repeat
from glob import glob
import cachetools
import numpy_indexed
import pickle
import warnings
from numbers import Integral

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
nd22 = nd2reader.ND2Reader("/home/jqs1/scratch/fidelity/all/180405_txnerr002.nd2")

In [ ]:
b"Dimensions: T(32) x XY(1040) x \xce\xbb(2)".decode("utf8")

In [ ]:
nd2._parser._calculate_image_group_number(3, 2, 0)

In [ ]:
nd22._parser

In [ ]:
nd2 = nd2reader.ND2Reader(
    "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2"
)

In [ ]:
s = nd2._parser._raw_metadata._parse_dimension_text().decode("utf8")

In [ ]:
nd22._parser._calculate_image_group_number(2, 3, 0)

In [ ]:
s2 = "Dimensions: T(32) x XY(1040) x λ(2)"

In [ ]:
np.product(np.array(list(dims.values())))

In [ ]:
dims

In [ ]:
key = {"T": 2, "XY": 3, "λ": 0}

In [ ]:
2 * 1040 + 3

In [ ]:
s

In [ ]:
nd2._parser._raw_metadata.image_attributes

In [ ]:
get_nd2_frame(
    "/home/jqs1/scratch/fidelity/190311/190311_205a_10ms_laser10pct_000.nd2", 0, 0, 0
)

In [ ]:
get_nd2_frame(
    "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2", 0, 0, 111
)

In [ ]:
plt.figure(figsize=(10, 10))
f = nd2.get_frame(667)
plt.imshow(f)

In [ ]:
plt.figure(figsize=(10, 10))
f = get_nd2_frame(
    "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2", 0, 0, 667
)
plt.imshow(f)

In [ ]:
nd2 = get_nd2_reader(
    "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2"
)

In [ ]:
plt.figure(figsize=(10, 10))
f = nd2.get_frame_2D(v=0, c=0, t=667)
plt.imshow(f)

In [ ]:
nd2.metadata

In [ ]:
plt.figure(figsize=(10, 10))
f = nd2.get_frame_2D(v=0, c=0, t=666, z=0)
plt.imshow(f)

In [ ]:
fs = [
    get_nd2_frame(
        "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2", 0, 0, t
    )
    for t in range(995, 999)
]

In [ ]:
hv.HoloMap({t: RevImage(img) for t, img in enumerate(fs)})

In [ ]:
f1 = get_nd2_frame(
    "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2", 0, 0, 0
)
f2 = get_nd2_frame(
    "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2", 0, 0, 999
)
f1 = f1 / np.nanmax(f1)
f2 = f2 / np.nanmax(f2)
z = f1 - f2

In [ ]:
%%output size=250
RevImage(z)

In [ ]:
hv.extension("bokeh")

In [ ]:
def RevImage(img, **kwargs):
    return _RevImage(hv.Image, img, **kwargs)


def _RevImage(cls, img, **kwargs):
    return cls(img[::-1], bounds=(0, 0, img.shape[1], img.shape[0])).options(
        invert_yaxis=True
    )

In [ ]:
def hessian_eigenvalues(img):
    I = skimage.filters.gaussian(img, 1.5)
    I_x = skimage.filters.sobel_h(I)
    I_y = skimage.filters.sobel_v(I)
    I_xx = skimage.filters.sobel_h(I_x)
    I_xy = skimage.filters.sobel_v(I_x)
    I_yx = skimage.filters.sobel_h(I_y)
    I_yy = skimage.filters.sobel_v(I_y)
    kappa_1 = (I_xx + I_yy) / 2
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", RuntimeWarning)
        kappa_2 = (np.sqrt((I_xx + I_yy) ** 2 - 4 * (I_xx * I_yy - I_xy * I_yx))) / 2
    k1 = kappa_1 + kappa_2
    k2 = kappa_1 - kappa_2
    k1[np.isnan(k1)] = 0
    k2[np.isnan(k2)] = 0
    return k1, k2


def invert(img):
    img2 = img.astype(np.float32).copy()
    img2 = np.median(img2) - img2
    img2[img2 < 0] = 0
    return img2


from cytoolz import reduce


def repeat_apply(func, n):
    if n <= 0:
        return lambda x: x
    return reduce(lambda f1, f2: compose(f1, f2), [func] * n)


def normalize_componentwise(
    img,
    img_labels,
    weighted=False,
    label_index=None,
    dilation=5,
    in_place=False,
    dtype=np.float32,
):
    if not in_place:
        img = img.astype(dtype).copy()
    if label_index is None:
        label_index = np.unique(img_labels)
    maxes = scipy.ndimage.maximum(img, labels=img_labels, index=label_index)
    if weighted:
        median = np.median(maxes[1:])  # exclude background from median
    img_labels = repeat_apply(skimage.morphology.dilation, dilation)(img_labels)
    img[img_labels == 0] = 0
    for idx, label in enumerate(label_index):
        mask = img_labels == label
        if weighted:
            img[mask] *= min(maxes[idx] / median, 1)
        else:
            img[mask] /= maxes[idx]
    return img

In [ ]:
DEFAULT_REGIONPROPS = [
    "area",
    "centroid",
    "eccentricity",
    "min_intensity",
    "mean_intensity",
    "max_intensity",
    "major_axis_length",
    "minor_axis_length",
    "orientation",
    "perimeter",
    "solidity",
    "weighted_centroid",
]


def get_regionprops(label_image, intensity_image, properties=DEFAULT_REGIONPROPS):
    rps = skimage.measure.regionprops(
        label_image, intensity_image, coordinates="rc", cache=False
    )
    if not len(rps):
        return None
    cols = {prop: [getattr(rp, prop) for rp in rps] for prop in properties}
    for col, values in list(cols.items()):
        if isinstance(values[0], tuple):
            del cols[col]
            # TODO: store coordinates as multiindex?
            cols[col + "_x"] = [v[0] for v in values]
            cols[col + "_y"] = [v[1] for v in values]
    df = pd.DataFrame(cols, index=range(1, len(rps) + 1))
    df.index.name = "label"
    return df

In [ ]:
img = bf[:600, :600]
img = invert(img)
# img = skimage.transform.pyramid_expand(img, upscale=2, multichannel=False)
mask = img > skimage.filters.threshold_otsu(img)
img_labels = skimage.morphology.label(mask)
img = normalize_componentwise(img, img_labels)

In [ ]:
def segment(img):
    img_frangi = skimage.filters.frangi(img, scale_range=(0.1, 1.5), scale_step=0.1)
    del img
    mask = img_frangi < np.percentile(img_frangi, 90)
    del img_frangi
    mask = skimage.segmentation.clear_border(mask)
    mask = repeat_apply(skimage.morphology.erosion, 2)(mask)
    mask = skimage.morphology.remove_small_objects(mask, 5)
    labels = skimage.measure.label(mask)
    # TODO: not even watershedding??
    return labels

In [ ]:
ND2READER_CACHE = cachetools.LFUCache(maxsize=48)


def _get_nd2_reader(filename, memmap=False, **kwargs):
    return nd2reader.ND2Reader(filename, memmap=memmap, **kwargs)


# get_nd2_reader = cachetools.cached(cache=ND2READER_CACHE)(_get_nd2_reader)
get_nd2_reader = _get_nd2_reader

# TODO: modified
def get_nd2_frame(filename, position, channel, t, memmap=False):
    reader = get_nd2_reader(filename, memmap=memmap)
    if not isinstance(channel, Integral):
        channel = reader.metadata["channels"].index(channel)
    print(">", dict(v=position, c=channel, t=t, memmap=memmap))
    ary = reader.get_frame_2D(v=position, c=channel, t=t, memmap=memmap)
    return ary


def nd2_to_futures(client, filename):
    nd2 = get_nd2_reader(filename)
    frames = [client.submit(get_nd2_frame, filename, t) for t in range(nd2.sizes["t"])]
    return frames


# TODO: changed argument order
def map_over_labels(func, label_image, intensity_image):
    # assumes are consecutive integers 1,...,N
    groups = numpy_indexed.group_by(
        label_image.ravel(), intensity_image.ravel(), reduction=func
    )
    return [g[1] for g in groups]


def nd2_to_dask(filename, position, channel):
    nd2 = get_nd2_reader(filename)
    frame0 = get_nd2_frame(filename, position, channel, 0)
    frames = [
        dask.delayed(get_nd2_frame)(filename, position, channel, t)
        for t in range(nd2.sizes["t"])
    ]
    arrays = [
        da.from_delayed(frame, dtype=frame0.dtype, shape=frame0.shape)
        for frame in frames
    ]
    stack = da.stack(arrays, axis=0)
    return stack


def nd2_to_futures(client, filename, position, channel):
    nd2 = get_nd2_reader(filename)
    frame0 = get_nd2_frame(filename, position, channel, 0)
    frames = [
        client.submit(get_nd2_frame, filename, position, channel, t)
        for t in range(nd2.sizes["t"])
    ]
    return frames


def process_file(
    client,
    col_to_funcs,
    photobleaching_filename,
    segmentation_filename=None,
    initial_filename=None,
    final_filename=None,
    time_slice=slice(None),
):
    nd2 = get_nd2_reader(photobleaching_filename)
    num_positions = nd2.sizes["v"]
    del nd2
    data = {}
    for position in range(num_positions):
        # TODO: why won't dask array work?
        # photobleaching_frames = nd2_to_dask(photobleaching_filename, position, 0)
        photobleaching_frames = nd2_to_futures(
            client, photobleaching_filename, position, 0
        )
        if segmentation_filename:
            segmentation_frame = client.submit(
                get_nd2_frame, segmentation_filename, position, 0, 0
            )
        else:
            segmentation_filename = photobleaching_filename
            segmentation_frame = photobleaching_frames[0]
        segmentation_nd2 = get_nd2_reader(segmentation_filename)
        if segmentation_nd2.metadata["channels"][0] == "BF":
            segmentation_func = compose(segment, invert)
        else:
            segmentation_func = segment
        # TODO: assuming segmenting in phase
        labels = client.submit(segmentation_func, segmentation_frame)
        sandwich_frames = []
        if initial_filename is not None:
            initial_frame = client.submit(
                get_nd2_frame, initial_filename, position, 0, 0
            )
            sandwich_frames.append(initial_frame)
            regionprops_frame = initial_frame
        else:
            regionprops_frame = segmentation_frame
        if final_filename is not None:
            sandwich_frames.append(
                client.submit(get_nd2_frame, final_filename, position, 0, 0)
            )
        sandwich_traces = {
            col: client.submit(
                np.transpose,
                [
                    client.submit(map_over_labels, func, labels, frame)
                    for frame in sandwich_frames
                ],
            )
            for col, func in col_to_funcs.items()
        }
        regionprops = client.submit(get_regionprops, labels, regionprops_frame)
        traces = {
            col: client.submit(
                np.transpose,
                [
                    client.submit(map_over_labels, func, labels, frame)
                    for frame in photobleaching_frames[time_slice]
                ],
            )
            # traces = {col: client.submit(np.transpose, client.map(partial(map_over_labels, func, labels),
            #                                                      photobleaching_frames[time_slice]))
            for col, func in col_to_funcs.items()
        }
        data[position] = {
            "regionprops": regionprops,
            "sandwich_traces": sandwich_traces,
            "traces": traces,
            "labels": labels,
            "segmentation_frame": segmentation_frame,
        }
    return data

# Run

In [ ]:
dask.config.config["distributed"]["scheduler"]["allowed-failures"] = 20
# dask.config.config['distributed']['worker']['memory'] = {'target': 0.4,
#                                                         'spill': 0.5,
#                                                         'pause': 0.9,
#                                                         'terminate': 0.95}

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="03:00:00",
    memory="8GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/projects/molecule-counting/log",
    cores=1,
    processes=1,
)
# diagnostics_port=('127.0.0.1', 8787),
# env_extra=['export PYTHONPATH=\"/home/jqs1/projects/matriarch\"'])
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(10)

## Run

In [ ]:
funcs = {"mean": np.mean, "median": np.median}
#'p0.05': partial(np.percentile, q=5),
#'p0.20': partial(np.percentile, q=20),
#'p0.70': partial(np.percentile, q=70),
#'p0.95': partial(np.percentile, q=95)}

In [ ]:
fluorescence_filenames = glob("/home/jqs1/paulsson/fluorescence/*.nd2")
phase_filenames = glob("/home/jqs1/paulsson/phase/*_0001.nd2")
sandwich_filenames = glob("/home/jqs1/paulsson/sandwich/*_0001.nd2")

In [ ]:
# dark_frames = nd2_to_dask('/n/scratch2/jqs1/fidelity/190313/calibration/dark_100ms.nd2', 0, 'Dark')
# dark_frame = dark_frames.mean(axis=0)
# dark_std = dark_frames.std(axis=0)

In [ ]:
# gfp_flat = nd2_to_dask('/n/scratch2/jqs1/fidelity/190313/calibration/190313_flatfield_FITC_cfpslide_100ms_20pct.nd2').median(axis=0)
# mcherry_flat = nd2_to_dask('/n/scratch2/jqs1/fidelity/190313/calibration/190313_flatfield_mcherry_gfpslide_100ms_000.nd2').median(axis=0)
# mcherry_flat_lowpower = nd2_to_dask('/n/scratch2/jqs1/fidelity/190313/calibration/190313_flatfield_mcherry_30ms_0.35pct_laser.nd2').median(axis=0)

In [ ]:
flats = {"GFP": gfp_flat, "MCHERRY": mcherry_flat}

In [ ]:
%%time
data = {}
for photobleaching_filename in fluorescence_filenames[:1]:
    data[photobleaching_filename] = process_file(
        client, funcs, photobleaching_filename, time_slice=slice(0, 10)
    )

for photobleaching_filename in phase_filenames[:0]:
    segmentation_filename = photobleaching_filename.replace("_0001.nd2", ".nd2")
    data[segmentation_filename] = process_file(
        client,
        funcs,
        photobleaching_filename,
        segmentation_filename=segmentation_filename,
    )

for initial_filename in sandwich_filenames[:0]:
    segmentation_filename = initial_filename.replace("_0001.nd2", ".nd2")
    photobleaching_filename = initial_filename.replace("_0001.nd2", "_0002.nd2")
    final_filename = initial_filename.replace("_0001.nd2", "_0003.nd2")
    data[segmentation_filename] = process_file(
        client,
        funcs,
        photobleaching_filename,
        segmentation_filename=segmentation_filename,
        initial_filename=initial_filename,
        final_filename=final_filename,
    )

In [ ]:
%%time
a = client.gather(data)

In [ ]:
a["/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2"][0]["regionprops"]

In [ ]:
ary = nd2_to_dask(
    "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2", 0, 0
)

In [ ]:
get_nd2_frame(
    "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2", 0, 0, 111
)

In [ ]:
for i in range(500):
    print(">", i)
    ary[i, 0, 0].compute()

In [ ]:
%%time
a = ary[:500].mean(axis=0).compute()

In [ ]:
%%time
b = ary[500:].mean(axis=0).compute()

In [ ]:
RevImage(b)

In [ ]:
%%output size=150
(
    RevImage(
        get_nd2_frame(
            "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2", 0, 0, 0
        )
    )
    + RevImage(
        get_nd2_frame(
            "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2", 0, 0, 1
        )
    )
    + RevImage(
        get_nd2_frame(
            "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2",
            0,
            0,
            500,
        )
    ).redim(z="z3")
)

In [ ]:
%%output size=250
RevImage(
    get_nd2_frame(
        "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_50pct_laser.nd2", 0, 0, 0
    )
)

In [ ]:
filename = "/home/jqs1/paulsson/fluorescence/190313_GFP_100ms_100pct_laser.nd2"
pos = 1
RevImage(a[filename][pos]["labels"]) + RevImage(
    a[filename][pos]["segmentation_frame"]
).redim(z="z2")

In [ ]:
bf = ND2Reader(
    "/home/jqs1/paulsson/190313_SB1_in_GFP_bleaching_in_CFP_100ms_100pct_laser_again_again.nd2"
).get_frame_2D()

In [ ]:
fitc = nd2_to_dask(
    "/home/jqs1/paulsson/190313_SB1_in_GFP_bleaching_in_CFP_100ms_100pct_laser_again_again_0001.nd2",
    0,
    "FITC",
)

In [ ]:
fitc_frame = fitc.mean(axis=0).compute()

In [ ]:
%%time
data_futures = {
    filename: process_file(client, filename, funcs) for filename in filenames
}

In [ ]:
%%time
data = client.gather(data_futures)

In [ ]:
with open("190313photobleaching.pickle", "wb") as f:
    pickle.dump(data, f)

# Plotting

In [ ]:
with open("190311photobleaching.pickle", "rb") as f:
    trace_res = pickle.load(f)

In [ ]:
list(trace_res.keys())

In [ ]:
b = trace_res["/n/scratch2/jqs1/fidelity/190311/190311_205a_10ms_laser10pct_001.nd2"]

In [ ]:
plt.plot(np.log(b[0]["mean"][10]))

In [ ]:
b[1].hist("area", bins=100)

In [ ]:
traces = b[0]["mean"]

In [ ]:
idxs = np.random.permutation(traces.shape[0])

In [ ]:
%%output size=250
curves = [
    {
        "x": np.arange(traces.shape[1]),
        "y": np.log(traces[i] / traces[i, 0]),
        "i": idxs[i],
    }
    for i in range(traces.shape[0] // 10)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20")

In [ ]:
%%output size=250
hv.Path((np.arange(traces.shape[1]), traces.T))

In [ ]:
%%output size=250
hv.Overlay.from_values([hv.Curve(t) for t in traces])

In [ ]:
%%output size=250
hv.Image(img) + hv.Image(segment(img))